In [1]:
import torch_geometric

import numpy as np
import pandas as pd
import torch
from torch.nn import Linear, LayerNorm, ReLU, Dropout
import torch.nn.functional as F
from torch_geometric.nn import ChebConv, NNConv, DeepGCNLayer, GATConv, DenseGCNConv, GCNConv, GraphConv
from torch_geometric.data import Data, DataLoader
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import scipy.sparse as sp

import warnings
warnings.filterwarnings("ignore")

# ref: https://medium.com/stanford-cs224w/fraud-detection-with-gat-edac49bda1a0

#### Import dataset

In [2]:
# import data 
df_features = pd.read_csv('data/elliptic_txs_features.csv', header=None)
df_edges = pd.read_csv("data/elliptic_txs_edgelist.csv")
df_classes =  pd.read_csv("data/elliptic_txs_classes.csv")

df_classes['class'] = df_classes['class'].map({'unknown': 2, '1':1, '2':0})

# merging dataframes
df_merge = df_features.merge(df_classes, how='left', right_on="txId", left_on=0)
df_merge.drop(0, axis=1, inplace=True)

# check if there are duplicate txId
print("Number of duplicate txId: ", df_merge.duplicated(subset=['txId']).sum())


Number of duplicate txId:  0


In [3]:
# rename column 0 to time_step
df_merge.rename(columns={1: 'time_step'}, inplace=True)
display(df_merge.head())
display(df_edges.shape)

,time_step,2,3,4,5,6,7,8,9,10,...,159,160,161,162,163,164,165,166,txId,class
0,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,-0.167933,...,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,230425980,2
1,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,-0.167948,...,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,5530458,2
2,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,-0.168576,...,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792,232022460,2
3,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,-0.115831,...,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,232438397,0
4,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,0.041399,...,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117,230460314,2


(234355, 2)

#### Split dataset masks

In [4]:
edges = df_edges.copy()

# Setup trans ID to node ID mapping
nodes = df_merge['txId'].values
map_id = {j:i for i,j in enumerate(nodes)} # mapping nodes to indexes

# Map transction IDs to node Ids
edges.txId1 = edges.txId1.map(map_id) #get nodes idx1 from edges list and filtered data
edges.txId2 = edges.txId2.map(map_id)
edges = edges.astype(int)

# Reformat and convert to tensor
edge_index = np.array(edges.values).T 
edge_index = torch.tensor(edge_index, dtype=torch.long).contiguous()

print("shape of edge index is {}".format(edge_index.shape))

shape of edge index is torch.Size([2, 234355])


In [5]:
node_features = df_merge.drop(['txId'], axis=1).copy()
print("unique=",node_features["class"].unique())

# Retain known vs unknown IDs
all_classified_idx = node_features['class'].loc[node_features['class']!=2].index # filter on known labels
all_unclassified_idx = node_features['class'].loc[node_features['class']==2].index
all_classified_illicit_idx = node_features['class'].loc[node_features['class']==1].index # filter on illicit labels
all_classified_licit_idx = node_features['class'].loc[node_features['class']==0].index # filter on licit labels

# node_features = node_features.drop(columns=[0, 1, 'class'])
display(node_features.head())

unique= [2 0 1]


,time_step,2,3,4,5,6,7,8,9,10,...,158,159,160,161,162,163,164,165,166,class
0,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,-0.167933,...,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
1,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,-0.167948,...,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,2
2,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,-0.168576,...,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792,2
3,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,-0.115831,...,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,0
4,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,0.041399,...,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117,2


In [6]:
train_classified_idx = node_features.loc[(node_features['time_step'] <= 34) & (node_features['class'] != 2)].index
test_classified_idx = node_features.loc[(node_features['time_step'] > 34) & (node_features['class'] != 2)].index
print("train_classified_idx.shape=",train_classified_idx.shape)
print("test_classified_idx.shape=",test_classified_idx.shape)

train_classified_idx.shape= (29894,)
test_classified_idx.shape= (16670,)


In [18]:
# save the train and test indices as csv, integer
np.savetxt("data/index/train_classified_idx.csv", train_classified_idx, delimiter=",", fmt='%d')
np.savetxt("data/index/test_classified_idx.csv", test_classified_idx, delimiter=",", fmt='%d')

In [8]:
node_features.drop(columns=['time_step'], inplace=True)
node_features.drop(columns=['class'], inplace=True)

# Convert to tensor
node_features_t = torch.tensor(np.array(node_features.values, dtype=np.double), dtype=torch.double)

In [9]:
# Define labels
labels = df_merge['class'].values

#create weights tensor with same shape of edge_index
weights = torch.tensor([1]* edge_index.shape[1] , dtype=torch.double) 

# Do train test split on classified_ids
train_idx = train_classified_idx
test_idx = test_classified_idx

# Create pyG dataset
data_graph = Data(x=node_features_t.float(), edge_index=edge_index, edge_attr=weights, 
                               y=torch.tensor(labels, dtype=torch.long))

# Add in the train and valid idx
data_graph.train_idx = train_idx
data_graph.test_idx = test_idx
data_graph

Data(x=[203769, 165], edge_index=[2, 234355], edge_attr=[234355], y=[203769], train_idx=Index([     3,      9,     10,     11,     16,     17,     25,     27,     29,
           30,
       ...
       136232, 136233, 136234, 136236, 136239, 136241, 136243, 136249, 136250,
       136258],
      dtype='int64', length=29894), test_idx=Index([136276, 136277, 136278, 136279, 136280, 136282, 136285, 136287, 136288,
       136291,
       ...
       203727, 203730, 203736, 203740, 203750, 203752, 203754, 203759, 203763,
       203766],
      dtype='int64', length=16670))

#### GCN Model and Training

In [10]:
# 2-layer GCN
class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes, hidden_channels=128):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)
        self.dropout = Dropout(0.5)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return x

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [12]:
def train(model, data, optimizer):
    model.train()
    data = data.to(device)
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    # out = out.reshape((data.x.shape[0]))
    # TODO :use weighted cross entropy loss
    loss = F.cross_entropy(out[data.train_idx], data.y[data.train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()

def test(model, data):
    model.eval()
    with torch.no_grad():
        data = data.to(device)
        out = model(data.x, data.edge_index)
        # out = out.reshape((data.x.shape[0]))
        pred_scores = out[data.test_idx]
        # print("pred_scores=",pred_scores)
        pred = torch.argmax(pred_scores, dim=1)
        # print("pred=",pred)
        y = data.y[data.test_idx]
        # print("y=",y.shape)
        acc = accuracy_score(y.cpu(), pred.cpu())
        return acc

In [13]:
num_features = data_graph.num_node_features
print("num_features=",num_features)

num_features= 165


In [14]:
model = GCN(num_features, 2).to(device)
num_epochs = 200
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(num_epochs+1):
    loss = train(model, data_graph, optimizer)
    acc = test(model, data_graph)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Accuracy: {acc:.4f}')

Epoch: 000, Loss: 2.3206, Accuracy: 0.4989
Epoch: 010, Loss: 0.4902, Accuracy: 0.4610
Epoch: 020, Loss: 0.4128, Accuracy: 0.4731
Epoch: 030, Loss: 0.3578, Accuracy: 0.4868
Epoch: 040, Loss: 0.3223, Accuracy: 0.4933
Epoch: 050, Loss: 0.3063, Accuracy: 0.5012
Epoch: 060, Loss: 0.2973, Accuracy: 0.5002
Epoch: 070, Loss: 0.2909, Accuracy: 0.5044
Epoch: 080, Loss: 0.2813, Accuracy: 0.5184
Epoch: 090, Loss: 0.2773, Accuracy: 0.5289
Epoch: 100, Loss: 0.2686, Accuracy: 0.5413
Epoch: 110, Loss: 0.2619, Accuracy: 0.5499
Epoch: 120, Loss: 0.2570, Accuracy: 0.5592
Epoch: 130, Loss: 0.2554, Accuracy: 0.5690
Epoch: 140, Loss: 0.2512, Accuracy: 0.5775
Epoch: 150, Loss: 0.2483, Accuracy: 0.5849
Epoch: 160, Loss: 0.2418, Accuracy: 0.5910
Epoch: 170, Loss: 0.2407, Accuracy: 0.5996
Epoch: 180, Loss: 0.2398, Accuracy: 0.6100
Epoch: 190, Loss: 0.2371, Accuracy: 0.6179
Epoch: 200, Loss: 0.2346, Accuracy: 0.6305


In [15]:
# test accuracy
test_acc = test(model, data_graph)
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.6305
